In [46]:
import math
from math import gcd
import numpy as np
import itertools

In [16]:
class TaskSet:
    
    
    def __init__(self):
        
        # Initializing empty local array of tasks array
        self.__tasks = []

        
    def addTask(self, period, deadline, computeTime, phase):
        
        # Adding a task to the array of tasks
        self.__tasks.append({'period': period, 'deadline': deadline, 'computeTime': computeTime, 'phase': phase})

        
    def removeTask(self, index):
        
        # Deleting a task at index
        del self.__tasks[index]

        
    def getTask(self, index):
        
        # Return a task at certain index
        return self.__tasks[index]

    
    def getAllTasks(self):
        
        # Return all tasks
        return self.__tasks

    
    def size(self):
        
        # Returns number of tasks in the taskset
        return len(self.__tasks)
    
    
    def updatePhases(self, phase_array):
        
        # Updating phases of all tasks
        for i in range(len(phase_array)):
            self.__tasks[i]['phase'] = phase_array[i]

            
            

In [132]:
class CyclicExecutiveSchedule:
    
    
    def __init__(self, task_set):
        
        # Creating local copy of task_set object
        self.task_set = task_set
        
        # Calculating hyperperiod for the given task_set
        self.hyperperiod = self.compute_hyperperiod()
        
        # Cacluating all valid frame sizes
        self.valid_frames = self.compute_valid_frames()
        
        # Making an array of periods of all tasks in the taskset
        periods = [task['period'] for task in self.task_set.getAllTasks()]
        
        # Making an array of compute times of all tasks in the taskset
        compute_times = [task['computeTime'] for task in self.task_set.getAllTasks()]
        
        # Computing utilization factor
        self.utilization = 0
        for i in range(len(compute_times)):
            self.utilization += (compute_times[i] / periods[i])
        
        # Basic check that utilization factor does not exceed 100%, otherwise give warning
        if self.utilization>1:
            print("Utilization is "+str(self.utilization*100)+" % and It cannot be scheduled.")
        
        
    def compute_hyperperiod(self):
        
        # Making an array of periods of all tasks in the taskset
        periods = [task['period'] for task in self.task_set.getAllTasks()]
        
        # Returning hyperperiod of all tasks in the taskset
        return math.lcm(*periods)

    
    def compute_valid_frames(self):
        
        # Making an array of periods of all tasks in the taskset
        periods = [task['period'] for task in self.task_set.getAllTasks()]
        
        # Making an array of compute times of all tasks in the taskset
        compute_times = [task['computeTime'] for task in self.task_set.getAllTasks()]
        
        # Finding out the minimum time period of a task in the taskset
        min_period = min(periods)
        
        # Finding out the maximum time period of a task in the taskset
        max_compute = max(compute_times)
        
        # Making an array of valid frame sizes
        self.valid_frames = []
        print("Min period is: "+str(min_period))
        print("\n")
        print("Max compute is :"+str(max_compute))
        print("\n")
        
        # Adding valid frame sizes to the array of valid frames after meeting the criteria
        for frame_size in range(max_compute, min_period+1):
            if (frame_size >= max_compute)and(frame_size <= min_period)and(self.hyperperiod % frame_size == 0):
                is_valid = True
                for task in self.task_set.getAllTasks():
                    period = task['period']
                    deadline = task['deadline']
                    gcd_val = gcd(period, frame_size)
                    frame_gcd = 2 * frame_size - gcd_val
                    if frame_gcd > deadline:
                        is_valid = False
                        break
                if is_valid:
                    self.valid_frames.append(frame_size)
        self.valid_frames = list(set(self.valid_frames))
        
        # Returing valid frames array
        return self.valid_frames
    
    
    def generate_combinations(self, numbers, target_sum, required_numbers):
        
        ## This is a helping function that helps in scheduling more than one tasks in one frame ##
        
        # This function takes numbers like [1 7 2 3 4] and required_numbers [7]
        # Generates a combination of numbers that should be less than or equal to target_sum like 10
        # Combination should not miss required_numbers and ignores in case of empty
        # Combination should be as diverse as possible that target_sum should be sum as many members of the set as possible
        # Combination for this case would be [7 1 2] not like [7 3] because of less diversity or [1 2 3 4] because [7] is required
        
        result = []
        max_combination = ()
        max_num_unique_elements = 0
        
        for r in range(1, len(numbers) + 1):
            combinations = itertools.combinations(numbers, r)
            for combination in combinations:
                if all(num in combination for num in required_numbers):
                    current_sum = sum(combination)
                    if current_sum <= target_sum:
                        num_unique_elements = len(set(combination))
                        if num_unique_elements > max_num_unique_elements:
                            max_num_unique_elements = num_unique_elements
                            max_combination = combination
                        elif num_unique_elements == max_num_unique_elements and len(combination) > len(max_combination):
                            max_combination = combination
        return max_combination if max_combination else "Cannot be done"
    
    
    def get_matching_numbers(self, number_mask, number_array):
        
        ## This is a helping function
        
        # This function takes numbers array like [1 2 3 10 8]
        # A numbers mask like [True False False True False]
        # Returns [1 10]
        
        matching_numbers = []
        for i, value in enumerate(number_mask):
            if value:
                matching_numbers.append(number_array[i])
        return np.array(matching_numbers)
    
    
    def find_feasible_schedule(self):
        
        periods = [task['period'] for task in self.task_set.getAllTasks()]
        deadlines = [task['deadline'] for task in self.task_set.getAllTasks()]
        compute_times = [task['computeTime'] for task in self.task_set.getAllTasks()]
        # phase_times = [task['phase'] for task in self.task_set.getAllTasks()]
        
        for frame in self.valid_frames:
            #Dimensions [Frame][Instance][Task]
            self.block = np.full((int(self.hyperperiod/frame), int(self.hyperperiod/min(periods)), len(periods)), False, dtype=bool)
            print(self.block.shape)
            print("\n")
            for task_number in range(self.block.shape[2]):
                for task_instance in range(self.block.shape[1]):
                    for frame_number in range(self.block.shape[0]):
                        #print(str(frame_number)+str(task_number)+str(task_instance))
                        if((task_instance)*periods[task_number]<frame*(frame_number+1)<=(task_instance)*periods[task_number]+deadlines[task_number]):
                            self.block[frame_number, task_instance, task_number]=True
            ##Modification
            for frame_number in range(self.block.shape[0]):
                if (frame_number != self.block.shape[0]-1):
                    combination = self.generate_combinations(self.get_matching_numbers(np.logical_or.reduce(self.block[frame_number, :, :], axis=0), compute_times), frame, self.get_matching_numbers(np.logical_or.reduce(self.block[frame_number, :, :], axis=0)& ~np.logical_or.reduce(self.block[frame_number+1, :, :], axis=0), compute_times))
                else:
                    combination = self.generate_combinations(self.get_matching_numbers(np.logical_or.reduce(self.block[frame_number, :, :], axis=0), compute_times), frame, [])
                #Marking the task instance scheduled so that it does not schedule the same instance in next frame
                for index, compute_time in enumerate(compute_times):
                    for compute_scheduled in combination:
                        #print("Index is: "+str(index))
                        #print("Instance is: "+str(math.floor((frame*frame_number)/periods[index])))
                        #print("Statement: ")
                        #print((np.any(self.block[:, math.floor((frame*frame_number)/periods[index]), index])))
                        if (compute_time == compute_scheduled) and (np.any(self.block[:, math.floor((frame*frame_number)/periods[index]), index])):
                            print("Frame is "+str(frame_number)+" Compute scheduled is "+str(compute_scheduled)+" Task is "+str(index)+" Instance is "+str(math.ceil((frame*frame_number)/periods[index]))+" \n")
                            self.block[:, math.floor((frame*frame_number)/periods[index]), index] = np.full(self.block.shape[0], False, dtype=bool)
                            #break
                #print(combination)
                if(not(np.any(self.block))):
                    print("Your task is scheduled with frame size "+str(frame)+"\n")
                    break
                
            print(self.block)
            print("\n")
            
    

In [151]:
#Creating a taskset
my_task_set = TaskSet()

#my_task_set.addTask(period, deadline, compute time, phase)
my_task_set.addTask(20, 20, 2, 0)
my_task_set.addTask(20, 20, 1, 0)
my_task_set.addTask(4, 4, 1, 0)
my_task_set.addTask(7, 7, 2, 0)




schedule = CyclicExecutiveSchedule(my_task_set)
print("Hyperperiod:", schedule.hyperperiod)
print("Valid Frames:", schedule.valid_frames)


Min period is: 4


Max compute is :2


Hyperperiod: 140
Valid Frames: [2, 4]


In [152]:
schedule.find_feasible_schedule()

(70, 35, 4)


Frame is 0 Compute scheduled is 1 Task is 1 Instance is 0 

Frame is 0 Compute scheduled is 1 Task is 2 Instance is 0 

Frame is 1 Compute scheduled is 2 Task is 0 Instance is 1 

Frame is 1 Compute scheduled is 2 Task is 3 Instance is 1 

Frame is 2 Compute scheduled is 1 Task is 2 Instance is 1 

Frame is 4 Compute scheduled is 1 Task is 2 Instance is 2 

Frame is 5 Compute scheduled is 2 Task is 3 Instance is 2 

Frame is 6 Compute scheduled is 1 Task is 2 Instance is 3 

Frame is 7 Compute scheduled is 2 Task is 3 Instance is 2 

Frame is 8 Compute scheduled is 1 Task is 2 Instance is 4 

Frame is 10 Compute scheduled is 1 Task is 1 Instance is 1 

Frame is 10 Compute scheduled is 1 Task is 2 Instance is 5 

Frame is 11 Compute scheduled is 2 Task is 0 Instance is 2 

Frame is 11 Compute scheduled is 2 Task is 3 Instance is 4 

Frame is 12 Compute scheduled is 1 Task is 2 Instance is 6 

Frame is 14 Compute scheduled is 1 Task is 2 Instance is 7 

Frame is 15 Compute 